In [ ]:
from bs4 import BeautifulSoup as bs
import selenium
import unittest
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import io
import os
import time
import re

In [ ]:
def category_data(e_name,driver) :
    select = Select(driver.find_element_by_name(e_name))
    options = select.options
    df=[]
    for option in options:
        df.append(option.text)
    try:
        df.remove('Medical District *')
    except:
        df.remove('Speciality *')
    return df

In [ ]:
def get_div_data(source):
    soup = bs(source,'lxml')
    tag =soup.find_all('div','drdetails col-xs-12 col-sm-12 col-md-9 col-lg-9')
    print('\n___________\n{}'.format(tag))
    for fac in tag :
        with open("tag.txt", "a") as text_file:
            text_file.writelines(fac)   


In [ ]:
def get_selected_data(driver) :
    driver = webdriver.Chrome('C:/webdriver/bin/chromedriver.exe')
    driver.get('https://www.mohap.gov.ae/en/Pages/profdoctors.aspx')
    i=0
    source=""
    for dist in medical_district:
            for spec in speciality :
                try:
                    elem = driver.find_element_by_name('ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$ddlMedicalDistrict')
                    elem.send_keys(dist)
                   # wait= WebDriverWait(driver,20)
                    elem = driver.find_element_by_name('ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$ddlSpeciality')
                    elem.send_keys(spec)
                    elem1 = driver.find_element_by_name('ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$btnSearch')
                    wait= WebDriverWait(driver,5)
                    wait.until(EC.element_to_be_clickable((By.NAME,'ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$btnSearch')))
                    elem1.click()
                    time.sleep(4) 
                    s=driver.page_source
                    source += s
                   # elem = driver.find_element_by_class_name('drdetails col-xs-12 col-sm-12 col-md-9 col-lg-9')
                    s=driver.page_source
                    if s.find('No record found matching your criteria. Please refine your search.')>0:
                        print("found \n ")
                        continue
                    else:
                         print("####")
                    source += s
                    file_name="C:/Users/moamen_mohasseb/Doctor-data/data-files/{}_{}.txt".format(dist,spec)
                    with open(file_name, "w",encoding="utf8") as text_file:
                        text_file.write(s)
                  
                    i+=1
                    print('ok \n')
                    driver.refresh()

                except Exception as ex :
                    print('####exception#### ',ex)
                    continue
    driver.quit()

In [ ]:
driver = webdriver.Chrome('C:/webdriver/bin/chromedriver.exe')
driver.get('https://www.mohap.gov.ae/en/Pages/profdoctors.aspx')
e_name='ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$ddlMedicalDistrict'
select_df = pd.DataFrame()
medical_district=category_data(e_name,driver)
e_name='ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$ddlSpeciality' 
speciality=category_data(e_name,driver)
driver.quit()

In [ ]:
medical_district

### Get Data from site https://www.mohap.gov.ae/en/Pages/profdoctors.aspx 
 * crawling all expected result from the site eatch page in file 
 * get data from each file into pandas data frame 
 * export data to csv file

In [ ]:
get_selected_data(driver)

In [ ]:
all_data=[]
for url in os.listdir('C:/Users/moamen_mohasseb/Doctor-data/data-files/') :
    with open('C:/Users/moamen_mohasseb/Doctor-data/data-files/{}'.format(url), "r") as text_file:
        soup=bs(text_file.read(),'lxml')
        tags=soup.find_all('div','drdetails col-xs-12 col-sm-12 col-md-9 col-lg-9')
        for tag in tags :
            temp=tag.text.strip().replace('\n\n','|').replace('||','|')
            all_data.append(temp.replace('\n',''))
            print(temp.replace('\n',''),"\n new \n")

In [ ]:
all_data

In [ ]:
doctor_name_list=[]
medical_title_list=[]
Facility_Name_list=[]
Nationality_list=[]
for row in all_data :
    print(str(row))
    doctor_name=re.search(r'[a-zA-Z\-\s]+[a-zA-Z\-\s]+[a-zA-Z\-\s]+[|]',str(row)).group().split('|')[0]
    facility_name=re.search(r'Facility Name :[a-zA-Z0-9\s-]+\w*',str(row)).group().split('Facility Name :')[1]
    medical_title=re.search(r'Medical Title :[a-zA-Z0-9\s-]+\w*',str(row)).group().split('Medical Title :')[1]
    nationality=re.search(r'Nationality :[a-zA-Z0-9\s-]+\w*',str(row)).group().split('Nationality :')[1]
    print('\n',doctor_name,'\n',medical_title,'\n',facility_name,'\n',nationality,'\n')
    doctor_name_list.append(str(doctor_name).title())
    medical_title_list.append(medical_title)
    Facility_Name_list.append(facility_name)
    Nationality_list.append(nationality)
    

In [ ]:
doctor_df=pd.DataFrame()
doctor_df['Doctor_name']=doctor_name_list
doctor_df['Medical_title']=medical_title_list
doctor_df['Facility_Name']=Facility_Name_list
doctor_df['Nationality']=Nationality_list
doctor_df

In [ ]:
doctor_df.to_csv('C:/Users/moamen_mohasseb/Doctor-data/doctor_data.csv',index=0)